First:
- go to the terminal
- go to the `4_tuning_networks` folder, use `cd` to get there, or `z` to get there faster.
- if you are planning on closing the laptop to let this run by itself, start a tmux session by giving the command `tmux`. You can name the session with `ctrl+b $`, leave the tmux with `ctrl+b` d, access it again with `tmux a -t 0` or replace the 0 with the name you gave it. See [tmux](https://github.com/tmux/tmux/wiki/Getting-Started) for more info.
- Activate the `.venv`, and run `python hypertune.py`
- In the hypertune.py file, a tune_dir is specified: `models/ray`. We will check the contents of that folder after the hypertune finished. You can also use tensorboard to check the results. 
- this will take some time. I clocked 1901 seconds (30 minutes) on the VM

 0.0992908 |           119 |            3 |     26 |         576.744  |    0 |   0.942187

In [1]:
from pathlib import Path
tune_dir = Path("models/ray").resolve()
tune_dir.exists()

True

In [2]:
tunelogs = [d for d in tune_dir.iterdir()]
tunelogs.sort()
latest = tunelogs[-1]
latest

PosixPath('/Users/francesca/code_repo/MADS-exam-25/src/models/ray/train_2025-01-21_20-12-30')

In [3]:
from ray.tune import ExperimentAnalysis
import ray
ray.init(ignore_reinit_error=True)


analysis = ExperimentAnalysis(latest)


2025-01-21 20:40:34,366	INFO worker.py:1821 -- Started a local Ray instance.


So, we find some info:

In [4]:
analysis.results_df.columns


Index(['iterations', 'train_loss', 'test_loss', 'Accuracy', 'F1scoremicro',
       'F1scoremacro', 'Precisionmacro', 'Recallmacro', 'timestamp',
       'checkpoint_dir_name', 'done', 'training_iteration', 'date',
       'time_this_iter_s', 'time_total_s', 'pid', 'hostname', 'node_ip',
       'time_since_restore', 'iterations_since_restore', 'experiment_tag',
       'config/preprocessor', 'config/trainfile', 'config/testfile',
       'config/tune_dir', 'config/data_dir', 'config/batch', 'config/hidden',
       'config/dropout', 'config/num_layers', 'config/model_type',
       'config/num_blocks', 'config/num_classes', 'config/shape'],
      dtype='object')

Let's focus on the parameters we wanted to tune.

In [5]:
import plotly.express as px

plot = analysis.results_df
select = ["Accuracy", "config/hidden_size", "config/dropout", "config/num_layers"]
p = plot[select].reset_index().dropna()


ModuleNotFoundError: No module named 'plotly'

Let's sort by accuracy

In [6]:
p.sort_values("Accuracy", inplace=True)

NameError: name 'p' is not defined

Make a parallel plot

In [8]:
px.parallel_coordinates(p, color="Accuracy")


NameError: name 'px' is not defined

In [7]:
import seaborn as sns

sns.scatterplot(data=p, x="config/hidden_size", y="config/num_layers", hue="Accuracy", palette="coolwarm")

NameError: name 'p' is not defined

In [9]:
import matplotlib.pyplot as plt
cmap = sns.cubehelix_palette(as_cmap=True)
sns.scatterplot(data=p, x="config/hidden_size", y="config/num_layers", hue="Accuracy", palette="coolwarm")
sns.kdeplot(data=p, x="config/hidden_size", y="config/num_layers", cmap=cmap)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)

NameError: name 'p' is not defined

Get the best trial

In [10]:
analysis.get_best_trial(metric="test_loss", mode="min")


train_013366d5

The top ten

In [11]:
p[-10:]


NameError: name 'p' is not defined

Or the best config

In [12]:
analysis.get_best_config(metric="Accuracy", mode="max")


{'preprocessor': mltrainer.preprocessors.BasePreprocessor,
 'trainfile': PosixPath('/Users/francesca/code_repo/MADS-exam-25/data/processed/heart_big_oversampled_full_train.parq'),
 'testfile': PosixPath('/Users/francesca/code_repo/MADS-exam-25/data/processed/heart_big_test.parq'),
 'tune_dir': PosixPath('/Users/francesca/code_repo/MADS-exam-25/src/models/ray'),
 'data_dir': PosixPath('/Users/francesca/code_repo/MADS-exam-25/data/processed'),
 'batch': 32,
 'hidden': 57,
 'dropout': 0.0,
 'num_layers': 2,
 'model_type': '2DCNN',
 'num_blocks': 1,
 'num_classes': 5,
 'shape': [16, 12]}